In [1]:
using JuMP
using Ipopt
#using NLopt
using LinearAlgebra
using BlackBoxOptim

#using Optim
using HDF5

In [2]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

read_timeevolution (generic function with 1 method)

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [3]:
function infidelity_norm(ρ, σ)
    abs(1 - real(tr(ρ * σ)) - 2*sqrt(abs(det(ρ)*det(σ))))#^2
end

infidelity_norm (generic function with 1 method)

In [4]:
# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

Dc (generic function with 1 method)

### Kossakowski objective for two-level system

A linear operator $L: M(2) \rightarrow M(2)$ is the generator of a dynamical semigroup $t \rightarrow \Lambda_t$ of $M(2)$ iff it can be written in the form:

$L = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where

(i) $H = \sum_1^3 h_i f_i, \quad h_i \in \R$

(ii) $f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$


(iii) $\{C_{ij}\}= \begin{pmatrix} 
\gamma - 2\gamma_1    & - i a_3            &  i a_2 \\
i a_3                 & \gamma - 2\gamma_2 & -i a_1 \\
-i a_2                & i a_1              & \gamma - 2\gamma_3 
\end{pmatrix} =$

$=\begin{pmatrix} 
-\gamma_1 + \gamma_2 + \gamma_3   & - i a_3   &  i a_2 \\
i a_3                 & \gamma_1 - \gamma_2 + \gamma_3 & -i a_1 \\
-i a_2                & i a_1      & \gamma_1 + \gamma_2 - \gamma_3 
\end{pmatrix}$

In [5]:
# Define functions to construct H and C from parameters
function construct_H(ϵ, h_Re, h_Im)
    return [
             ϵ                     h_Re + im * h_Im
             h_Re - im * h_Im     -ϵ
           ] / 2
end

function construct_C(γ, a)
    return [
        -γ[1] + γ[2] + γ[3]  -im * a[3]            im * a[2]
        im * a[3]            γ[1] - γ[2] + γ[3]   -im * a[1]
        -im * a[2]           im * a[1]             γ[1] + γ[2] - γ[3]
    ]
end

# JuMP model setup
model = Model(Ipopt.Optimizer)

# Set options for higher precision
#set_optimizer_attribute(model, "tol", 1e-7)  # Convergence tolerance
#set_optimizer_attribute(model, "acceptable_tol", 1e-7)  # Acceptable tolerance
#set_optimizer_attribute(model, "constr_viol_tol", 1e-8)  # Constraint violation tolerance
#set_optimizer_attribute(model, "compl_inf_tol", 1e-8)  # Complementarity tolerance
#set_optimizer_attribute(model, "dual_inf_tol", 1e-8)  # Dual infeasibility tolerance

set_optimizer_attribute(model, "print_level", 5)


#model = Model(optimizer_with_attributes(NLopt.Optimizer, "algorithm" => :LD_SLSQP )) #:AUGLAG :LD_SLSQP :LD_MMA

# Define variables in JuMP
@variable(model, ϵ)
@variable(model, h_Re)
@variable(model, h_Im)
@variable(model, γ[1:3])
@variable(model, a[1:3])

# Define κ terms in JuMP
@expression(model, κ₁, -γ[1] + γ[2] + γ[3])
@expression(model, κ₂, γ[1] - γ[2] + γ[3])
@expression(model, κ₃, γ[1] + γ[2] - γ[3])

# Define the real and imaginary parts of Cˢʸᵐᵇ matrix in JuMP
@variable(model, C_Re[1:3, 1:3])
@variable(model, C_Im[1:3, 1:3])

# Define constraints for real and imaginary parts separately
@constraint(model, C_Re[1, 1] == κ₁)
@constraint(model, C_Re[1, 2] == 0)
@constraint(model, C_Re[1, 3] == 0)
@constraint(model, C_Re[2, 1] == 0)
@constraint(model, C_Re[2, 2] == κ₂)
@constraint(model, C_Re[2, 3] == 0)
@constraint(model, C_Re[3, 1] == 0)
@constraint(model, C_Re[3, 2] == 0)
@constraint(model, C_Re[3, 3] == κ₃)

@constraint(model, C_Im[1, 1] == 0)
@constraint(model, C_Im[1, 2] == -a[3])
@constraint(model, C_Im[1, 3] == a[2])
@constraint(model, C_Im[2, 1] == a[3])
@constraint(model, C_Im[2, 2] == 0)
@constraint(model, C_Im[2, 3] == -a[1])
@constraint(model, C_Im[3, 1] == -a[2])
@constraint(model, C_Im[3, 2] == a[1])
@constraint(model, C_Im[3, 3] == 0)

# Define constraints in JuMP
@constraint(model, ϵ >= 0)
@constraint(model, γ[1] >= 0)
@constraint(model, γ[2] >= 0)
@constraint(model, γ[3] >= 0)
@constraint(model, κ₁ + κ₂ + κ₃ >= 0)
@constraint(model, κ₁ >= 0)
@constraint(model, κ₂ >= 0)
@constraint(model, κ₃ >= 0)
@NLconstraint(model, κ₁ * κ₂ + κ₃ * κ₁ + κ₂ * κ₃ - a[1]^2 - a[2]^2 - a[3]^2 >= 0)
@NLconstraint(model, κ₁ * κ₂ * κ₃ - κ₁ * a[1]^2 - κ₂ * a[2]^2 - κ₃ * a[3]^2 >= 0)




((((-1.0 * γ[1] + γ[2] + γ[3]) * (γ[1] + -1.0 * γ[2] + γ[3]) * (γ[1] + γ[2] + -1.0 * γ[3]) - (-1.0 * γ[1] + γ[2] + γ[3]) * a[1] ^ 2.0) - (γ[1] + -1.0 * γ[2] + γ[3]) * a[2] ^ 2.0) - (γ[1] + γ[2] + -1.0 * γ[3]) * a[3] ^ 2.0) - 0.0 ≥ 0

In [6]:
# Define the objective function for optimization using RK4
function kossak_obj(ϵ, h_Re, h_Im, γ, a, ρ, t)
    H = construct_H(ϵ, h_Re, h_Im)
    C = construct_C(γ, a)

    # Define the Runge-Kutta 4th order method as an inner function
    function Dc_rk4(ρ, t, H, C, h)
        k1 = Dc(ρ, t, H, C)
        k2 = Dc(ρ + 0.5 * h * k1, t + 0.5 * h, H, C)
        k3 = Dc(ρ + 0.5 * h * k2, t + 0.5 * h, H, C)
        k4 = Dc(ρ + h * k3, t + h, H, C)
        return (k1 + 2*k2 + 2*k3 + k4) / 6
    end

    obj = 0.0
    for i in 2:length(ρ)
        ρ1 = ρ[i]
        h = t[i] - t[i - 1]
        ρ2 = ρ[i - 1] + h * Dc_rk4(ρ[i - 1], t[i - 1], H, C, h)
        obj += infidelity_norm(ρ1, ρ2)
    end
    return obj
end


kossak_obj (generic function with 1 method)

In [7]:
# Objective function

function kossak_obj_old(ϵ, h_Re, h_Im, γ, a, ρ, t)
    H = construct_H(ϵ, h_Re, h_Im)
    C = construct_C(γ, a)
    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) - 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += infidelity_norm(ρ1,ρ2)
    end
    return obj
end

# Define the objective function wrapper
function objectiveGEXY(ϵ, h_Re, h_Im, γ1, γ2, γ3, a1, a2, a3)
    γ = [γ1, γ2, γ3]
    a = [a1, a2, a3]
    objGEXY = kossak_obj(ϵ, h_Re, h_Im, γ, a, ρᵍ, tᵍ) + 
              kossak_obj(ϵ, h_Re, h_Im, γ, a, ρᵉ, tᵉ) + 
              kossak_obj(ϵ, h_Re, h_Im, γ, a, ρˣ, tˣ) + 
              kossak_obj(ϵ, h_Re, h_Im, γ, a, ρʸ, tʸ)
    return objGEXY
end


# Register the objective function with JuMP
register(model, :objectiveGEXY, 9, objectiveGEXY, autodiff = true)


In [8]:
# Define the density matrix evolution and time points

file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"
γᶠ = parse(ComplexF64, γᵢ)

tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γᵢ)
tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γᵢ)
tˣ, ρˣ = read_timeevolution(file_name, "B3", γᵢ)
tʸ, ρʸ = read_timeevolution(file_name, "B4", γᵢ);

In [9]:
# Define the objective function in JuMP
@NLobjective(model, Min, objectiveGEXY(ϵ, h_Re, h_Im, γ[1], γ[2], γ[3], a[1], a[2], a[3]))

# Optimize the model
JuMP.optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       33
Number of nonzeros in inequality constraint Jacobian.:       28
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       27
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       18
Total number of inequality co

In [11]:
# Get the optimized parameters
optimized_ϵ = value(ϵ)
optimized_h_Re = value(h_Re)
optimized_h_Im = value(h_Im)
optimized_γ = value.(γ)
optimized_a = value.(a)

# Substitute the optimized parameters back into H and C
optimized_H = construct_H(optimized_ϵ, optimized_h_Re, optimized_h_Im)
optimized_C = construct_C(optimized_γ, optimized_a)

┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP /home/zah/.julia/packages/JuMP/Gwn88/src/optimizer_interface.jl:761


OptimizeNotCalled: OptimizeNotCalled()

In [10]:
optimized_H

2×2 Matrix{ComplexF64}:
 -2.17442e-13+0.0im           8.07444e-5-0.000239661im
   8.07444e-5+0.000239661im  2.17442e-13+0.0im

In [26]:
function get_lindblad_operators(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end

get_lindblad_operators (generic function with 1 method)

In [27]:
Hˢⁱᵈ = convert.(ComplexF64,optimized_H)
Cˢⁱᵈ = convert.(ComplexF64,optimized_C)

effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

2-element Vector{Any}:
 ComplexF64[0.01837934870398452 + 0.0im -0.0036634172925965633 + 0.008969337099264502im; -0.009549260695663276 - 0.005338428187444248im -0.01837934870398452 + 0.0im]
 ComplexF64[0.012326227270335577 + 0.0im -0.007526236857586301 - 0.0017856474201574622im; 0.03376957992542453 + 0.02663278473166229im -0.012326227270335577 + 0.0im]

In [28]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [29]:
using QuantumOptics
basis = NLevelBasis(2)

include("../LiPoSID.jl")

using Statistics

In [30]:
FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert.(Float64, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = DenseOperator(basis,ρₛ[1])
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #print("effective_Lindblad_ops for Kossakowski")

    
    effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

    #print("Simulating Kossakowski")

    tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
    ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

    #print("Calculating Fidelity")

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println()
println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.13288614450649827
Median fidelity for 0.25133: 0.7668635472557752


D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 

Mimimal fidelity for 0.25133: 0.9995188642141652

Median fidelity for 0.25133: 0.9999049135214064